In [21]:
import pandas as pd
import numpy as np
import psycopg2
import pandas.io.sql as psql

from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

from sqlalchemy import create_engine


# Basic EDA-Understanding the datasets:

In [2]:
census_df=pd.read_csv('Census_Data_2010.csv', encoding='latin-1')
census_df.head()

,GIDTR,State,State_name,County,County_name,Tract,Flag,Num_BGs_in_Tract,LAND_AREA,AIAN_LAND,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,1.001020e+09,1.0,Alabama,1.0,Autauga County,20100.0,NaN,2.0,3.788,0.0,...,100.0,0.0,68.25,1.92,0.0,16.39,81.69,61.84,6.40,0.0
1,1.001020e+09,1.0,Alabama,1.0,Autauga County,20200.0,NaN,2.0,1.290,0.0,...,100.0,0.0,68.82,2.28,0.0,13.07,84.65,60.79,8.03,0.0
2,1.001020e+09,1.0,Alabama,1.0,Autauga County,20300.0,NaN,2.0,2.065,0.0,...,100.0,0.0,72.95,1.67,0.0,6.53,91.79,72.95,0.00,0.0
3,1.001020e+09,1.0,Alabama,1.0,Autauga County,20400.0,NaN,4.0,2.464,0.0,...,100.0,0.0,77.64,1.46,0.0,5.51,93.03,77.64,0.00,0.0
4,1.001020e+09,1.0,Alabama,1.0,Autauga County,20500.0,NaN,3.0,4.401,0.0,...,100.0,0.0,70.97,2.16,0.0,5.96,91.87,70.97,0.00,0.0


In [3]:
grad_rates=pd.read_csv('Graduation_Rates.csv', encoding='latin-1')
grad_rates.head()

,STNAM,FIPST,leaid11,leanm11,ALL_COHORT_1112,ALL_RATE_1112,MAM_COHORT_1112,MAM_RATE_1112,MAS_COHORT_1112,MAS_RATE_1112,...,MTR_COHORT_1112,MTR_RATE_1112,MWH_COHORT_1112,MWH_RATE_1112,CWD_COHORT_1112,CWD_RATE_1112,ECD_COHORT_1112,ECD_RATE_1112,LEP_COHORT_1112,LEP_RATE_1112
0,ALABAMA,1,100005,Albertville City,268,83,NaN,NaN,NaN,NaN,...,NaN,NaN,213.0,85-89,18.0,60-79,107.0,70-74,9.0,LT50
1,ALABAMA,1,100006,Marshall County,424,79,2.0,PS,1.0,PS,...,NaN,NaN,391.0,79,66.0,70-74,250.0,70-74,9.0,LT50
2,ALABAMA,1,100007,Hoover City,1042,91,1.0,PS,71.0,85-89,...,5.0,PS,688.0,94,63.0,70-74,216.0,75-79,21.0,21-39
3,ALABAMA,1,100008,Madison City,836,91,4.0,PS,44.0,GE90,...,2.0,PS,566.0,93,72.0,60-64,163.0,80-84,7.0,GE50
4,ALABAMA,1,100011,Leeds City,117,70-74,NaN,NaN,NaN,NaN,...,NaN,NaN,62.0,75-79,18.0,60-79,83.0,65-69,3.0,PS


In [4]:
grad_rates.columns

Index(['STNAM', 'FIPST', 'leaid11', 'leanm11', 'ALL_COHORT_1112',
       'ALL_RATE_1112', 'MAM_COHORT_1112', 'MAM_RATE_1112', 'MAS_COHORT_1112',
       'MAS_RATE_1112', 'MBL_COHORT_1112', 'MBL_RATE_1112', 'MHI_COHORT_1112',
       'MHI_RATE_1112', 'MTR_COHORT_1112', 'MTR_RATE_1112', 'MWH_COHORT_1112',
       'MWH_RATE_1112', 'CWD_COHORT_1112', 'CWD_RATE_1112', 'ECD_COHORT_1112',
       'ECD_RATE_1112', 'LEP_COHORT_1112', 'LEP_RATE_1112'],
      dtype='object')

In [5]:
overlap=pd.read_csv('Max_Tract_Overlap.csv', encoding='latin-1')
overlap.shape
overlap.head()

,Percentage,State,County,Tract Code,School District,District ID,Leaid,GIDTR,State.1,State_name,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,99.841328,1,45,20000,Fort Rucker School District,1,100001,1045020000,1,Alabama,...,100.0,0.0,63.78,2.15,0.0,18.88,78.97,55.58,8.20,0.0
1,96.376439,1,101,900,Maxwell AFB School District,3,100003,1101000900,1,Alabama,...,100.0,0.0,45.07,3.52,0.0,42.72,53.76,37.79,7.28,0.0
2,89.915474,1,95,30903,Albertville City School District,5,100005,1095030903,1,Alabama,...,100.0,0.0,63.14,2.48,0.0,10.39,87.13,55.74,7.40,0.0
3,100.000000,1,95,31000,Marshall County School District,6,100006,1095031000,1,Alabama,...,100.0,0.0,63.27,3.52,0.0,17.02,79.45,56.31,6.96,0.0
4,99.903457,1,73,14412,Hoover City School District,7,100007,1073014412,1,Alabama,...,100.0,0.0,81.77,1.97,0.0,2.98,95.06,81.77,0.00,0.0


In [6]:
#with pd.option_context('display.max_rows', 4, 'display.max_columns', 557):
#    print(overlap);
#Here I learned how to print all of the columns but I will be checking the 
#data dictionary included in theproject for more information.

In [6]:
mapping=pd.read_csv('SD_TRACT_MAPPING2010.csv', encoding='latin-1')
mapping.shape

(105148, 7)

In [7]:
mapping.head()

,Percentage,State,County,Tract Code,School District,District ID,Leaid
0,100.000000,1,5,950300,Barbour County School District,300,100300
1,99.646318,1,5,950900,Eufaula City School District,1410,101410
2,0.353682,1,5,950900,Barbour County School District,300,100300
3,94.564782,1,5,950800,Eufaula City School District,1410,101410
4,5.435218,1,5,950800,Barbour County School District,300,100300


In [8]:
mapping.columns

Index(['Percentage', 'State', 'County', 'Tract Code', ' School District',
       ' District ID', 'Leaid'],
      dtype='object')

In [9]:
census_df.head()

,GIDTR,State,State_name,County,County_name,Tract,Flag,Num_BGs_in_Tract,LAND_AREA,AIAN_LAND,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,1.001020e+09,1.0,Alabama,1.0,Autauga County,20100.0,NaN,2.0,3.788,0.0,...,100.0,0.0,68.25,1.92,0.0,16.39,81.69,61.84,6.40,0.0
1,1.001020e+09,1.0,Alabama,1.0,Autauga County,20200.0,NaN,2.0,1.290,0.0,...,100.0,0.0,68.82,2.28,0.0,13.07,84.65,60.79,8.03,0.0
2,1.001020e+09,1.0,Alabama,1.0,Autauga County,20300.0,NaN,2.0,2.065,0.0,...,100.0,0.0,72.95,1.67,0.0,6.53,91.79,72.95,0.00,0.0
3,1.001020e+09,1.0,Alabama,1.0,Autauga County,20400.0,NaN,4.0,2.464,0.0,...,100.0,0.0,77.64,1.46,0.0,5.51,93.03,77.64,0.00,0.0
4,1.001020e+09,1.0,Alabama,1.0,Autauga County,20500.0,NaN,3.0,4.401,0.0,...,100.0,0.0,70.97,2.16,0.0,5.96,91.87,70.97,0.00,0.0


# Postgres Merging

In [10]:
# I want to merge that datasets with ACGR and tract_mapping on the 'leaid', 
#Tract_mapping over the census on 'state', 'county', and 'tract'

connie = "host='localhost' port='5432'"

con = psycopg2.connect(connie) 
# Set this so the DB can be created without issues
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

# Cursors are how we interact with SQL, 
cur = con.cursor()
#cur.execute('''CREATE DATABASE project_3 ''') # create a database

con.close()  # close the connection

In [11]:
import sys
con = None
 
try:
    con = psycopg2.connect("host='localhost' dbname='project_3'  port='5432'")   
    cur = con.cursor()
    
except psycopg2.DatabaseError :
    # if a connection was established.
    if con:
        # In the event of an error, "rollback" the changes being made.
        # this way no half ass querys or inserts are done.
        con.rollback()
 
    print('Error ' )    
    sys.exit(1) # more or less application programming stuff.  
    # If there was an app connecting, it would return this to be interpretted
    # as seen fit.

finally:   # regardless of if try or except was performed.  
    # Close the connection if it was made.
    if con:
        con.close()

In [12]:
engine = create_engine('postgresql://@localhost:5432/project_3')

In [ ]:
#Now that I have downloaded all of the reguired SQL programs, I can convert my dataframes
#To SQL tables.

#grad_rates.to_sql("grad_rates", engine)


In [ ]:
#census_df.to_sql("census", engine)

In [68]:
mapping.to_sql("sd_tract_mapping2", engine)

In [ ]:
#overlap.to_sql("overlap", engine)

### Setting up the class

In [25]:
class my_database:
    # init works with needing the connection.
    def __init__(self, host, name, port):
        self.host = str(host)
        self.name = str(name)
        self.port = str(port)
        
        self.connection = "host='%s' dbname='%s'  port='%s'" % (self.host,self.name,self.port)
    #^here we are initiating the class. It is good that we are using the host, name, and port for these.
        
    def insert(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        con.commit()
        con.close()
        #Let's say we wanted to create a query, we can define methods to get information from the instance.
      
    def query(self, query):
        con = psycopg2.connect(self.connection)   
        cur = con.cursor()
        cur.execute(query)
        rows = cur.fetchall()
        #rows = cur.fetchone()
        #^changed from cur.fetchall()
        cols = [desc[0] for desc in cur.description]
        
        df = pd.DataFrame(data =rows, columns = cols)
        con.close()
        return df

In [18]:
#this instantiates a database connection to a class
project_3 = my_database('localhost','project_3','5432')

In [15]:
#Here is the database connection attribute
project_3.connection

"host='localhost' dbname='project_3'  port='5432'"

In [59]:
project_3.query(
'''SELECT sd.*
FROM sd_tract_mapping AS sd
LIMIT 3
'''
)
#gotta use those 3 quotations to use the multi-line

,index,Percentage,State,County,Tract Code,School District,District ID,Leaid
0,0,100.000000,1,5,950300,Barbour County School District,300,100300
1,1,99.646318,1,5,950900,Eufaula City School District,1410,101410
2,2,0.353682,1,5,950900,Barbour County School District,300,100300


# Merging the sd_tract with grad rates

In [33]:
project_3.insert('''
CREATE TABLE gr_tract_full AS
SELECT gr.*, "Percentage", "State", "County", "Tract Code"
FROM grad_rates AS gr
FULL JOIN sd_tract_mapping AS sd2
ON sd2."Leaid" = gr."leaid11";
''')
#This worked on March 6th after my presentation.

In [34]:
#con = psycopg2.connect("host='localhost' dbname='project_3'  port='5432'")   
#cur = con.cursor()
# errors I kept getting indicated that I needed to keep the connection open for this prode

gr_tract = psql.read_sql("SELECT * FROM gr_tract_full;", con)
gr_tract.shape

(106564, 29)

In [79]:
grad_rates.head(2)

,STNAM,FIPST,leaid11,leanm11,ALL_COHORT_1112,ALL_RATE_1112,MAM_COHORT_1112,MAM_RATE_1112,MAS_COHORT_1112,MAS_RATE_1112,...,MTR_COHORT_1112,MTR_RATE_1112,MWH_COHORT_1112,MWH_RATE_1112,CWD_COHORT_1112,CWD_RATE_1112,ECD_COHORT_1112,ECD_RATE_1112,LEP_COHORT_1112,LEP_RATE_1112
0,ALABAMA,1,100005,Albertville City,268,83,NaN,NaN,NaN,NaN,...,NaN,NaN,213.0,85-89,18.0,60-79,107.0,70-74,9.0,LT50
1,ALABAMA,1,100006,Marshall County,424,79,2.0,PS,1.0,PS,...,NaN,NaN,391.0,79,66.0,70-74,250.0,70-74,9.0,LT50


In [78]:
gr_tract_ni = gr_tract.drop(['index'], axis=1)
#to merge with over_tract, I need to drop the index column

# Merging the grad_rates with over_tracts:

In [75]:
project_3.insert('''
CREATE TABLE gr_over_tract AS
SELECT gr.*, ot.*
FROM grad_rates AS gr
FULL JOIN over_tracts AS ot
ON ot."Leaid" = gr."leaid11";
''')



ERROR:root:An unexpected error occurred while tokenizing input
The following traceback may be corrupted or invalid
The error message is: ('EOF in multi-line string', (1, 0))



ProgrammingError: column "index" specified more than once


# Now we merge the overlap data....

In [66]:
mapping['PencentageSD'] = mapping['Percentage']

In [63]:
overlap.head(3)

,Percentage,State,County,Tract Code,School District,District ID,Leaid,GIDTR,State.1,State_name,...,pct_TEA_MailOutMailBack_CEN_2010,pct_TEA_Update_Leave_CEN_2010,pct_Census_Mail_Returns_CEN_2010,pct_Vacant_CEN_2010,pct_Deletes_CEN_2010,pct_Census_UAA_CEN_2010,pct_Mailback_Count_CEN_2010,pct_FRST_FRMS_CEN_2010,pct_RPLCMNT_FRMS_CEN_2010,pct_BILQ_Mailout_count_CEN_2010
0,99.841328,1,45,20000,Fort Rucker School District,1,100001,1045020000,1,Alabama,...,100.0,0.0,63.78,2.15,0.0,18.88,78.97,55.58,8.20,0.0
1,96.376439,1,101,900,Maxwell AFB School District,3,100003,1101000900,1,Alabama,...,100.0,0.0,45.07,3.52,0.0,42.72,53.76,37.79,7.28,0.0
2,89.915474,1,95,30903,Albertville City School District,5,100005,1095030903,1,Alabama,...,100.0,0.0,63.14,2.48,0.0,10.39,87.13,55.74,7.40,0.0


In [47]:
gr_tract.columns

Index(['index', 'STNAM', 'FIPST', 'leaid11', 'leanm11', 'ALL_COHORT_1112',
       'ALL_RATE_1112', 'MAM_COHORT_1112', 'MAM_RATE_1112', 'MAS_COHORT_1112',
       'MAS_RATE_1112', 'MBL_COHORT_1112', 'MBL_RATE_1112', 'MHI_COHORT_1112',
       'MHI_RATE_1112', 'MTR_COHORT_1112', 'MTR_RATE_1112', 'MWH_COHORT_1112',
       'MWH_RATE_1112', 'CWD_COHORT_1112', 'CWD_RATE_1112', 'ECD_COHORT_1112',
       'ECD_RATE_1112', 'LEP_COHORT_1112', 'LEP_RATE_1112', 'Percentage',
       'State', 'County', 'Tract Code'],
      dtype='object')

In [73]:
project_3.insert('''
CREATE TABLE over_tracts AS 
SELECT over.*, sd."PencentageSD"
FROM overlap AS over
FULL JOIN sd_tract_mapping2 AS sd
ON over."Leaid" = sd."Leaid";
''')

In [74]:
# errors I kept getting indicated that I needed to keep the connection open for this prode

over_tracts = psql.read_sql("SELECT * FROM over_tracts;", con)
over_tracts.shape

(105148, 559)

In [51]:
#^coming back to this after Dave gets back to me...

In [80]:
con.close()